In [5]:
import findspark
findspark.init('/usr/local/spark-3.3.2-bin-hadoop3')

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.stat import Correlation
import pyspark.sql.functions as F

In [3]:
spark = SparkSession.builder.getOrCreate()

In [7]:
df = spark.read.csv("hdfs://localhost:9000/usr/heart.csv", inferSchema=True,
                   header=True)

In [8]:
df.show()

+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|     M|          ATA|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|
| 49|     F|          NAP|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|
| 37|     M|          ATA|      130|        283|        0|        ST|   98|             N|    0.0|      Up|           0|
| 48|     F|          ASY|      138|        214|        0|    Normal|  108|             Y|    1.5|    Flat|           1|
| 54|     M|          NAP|      150|        195|        0|    Normal|  122|             N|    0.0|      Up|           0|
| 39|     M|          NAP|      

In [9]:
df.select("HeartDisease").show()

+------------+
|HeartDisease|
+------------+
|           0|
|           1|
|           0|
|           1|
|           0|
|           0|
|           0|
|           0|
|           1|
|           0|
|           0|
|           1|
|           0|
|           1|
|           0|
|           0|
|           1|
|           0|
|           1|
|           1|
+------------+
only showing top 20 rows



In [11]:
df.count()

918

In [12]:
len(df.columns)

12

In [13]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- ChestPainType: string (nullable = true)
 |-- RestingBP: integer (nullable = true)
 |-- Cholesterol: integer (nullable = true)
 |-- FastingBS: integer (nullable = true)
 |-- RestingECG: string (nullable = true)
 |-- MaxHR: integer (nullable = true)
 |-- ExerciseAngina: string (nullable = true)
 |-- Oldpeak: double (nullable = true)
 |-- ST_Slope: string (nullable = true)
 |-- HeartDisease: integer (nullable = true)



In [14]:
df.describe().show()

23/04/30 15:11:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------+-------------+------------------+------------------+-------------------+----------+------------------+--------------+------------------+--------+-------------------+
|summary|               Age|Gender|ChestPainType|         RestingBP|       Cholesterol|          FastingBS|RestingECG|             MaxHR|ExerciseAngina|           Oldpeak|ST_Slope|       HeartDisease|
+-------+------------------+------+-------------+------------------+------------------+-------------------+----------+------------------+--------------+------------------+--------+-------------------+
|  count|               918|   918|          918|               918|               918|                918|       918|               918|           918|               918|     918|                918|
|   mean|53.510893246187365|  null|         null|132.39651416122004| 198.7995642701525|0.23311546840958605|      null|136.80936819172112|          null|0.8873638344226581|    null| 0.5533769063180

In [15]:
df.head(5)

[Row(Age=40, Gender='M', ChestPainType='ATA', RestingBP=140, Cholesterol=289, FastingBS=0, RestingECG='Normal', MaxHR=172, ExerciseAngina='N', Oldpeak=0.0, ST_Slope='Up', HeartDisease=0),
 Row(Age=49, Gender='F', ChestPainType='NAP', RestingBP=160, Cholesterol=180, FastingBS=0, RestingECG='Normal', MaxHR=156, ExerciseAngina='N', Oldpeak=1.0, ST_Slope='Flat', HeartDisease=1),
 Row(Age=37, Gender='M', ChestPainType='ATA', RestingBP=130, Cholesterol=283, FastingBS=0, RestingECG='ST', MaxHR=98, ExerciseAngina='N', Oldpeak=0.0, ST_Slope='Up', HeartDisease=0),
 Row(Age=48, Gender='F', ChestPainType='ASY', RestingBP=138, Cholesterol=214, FastingBS=0, RestingECG='Normal', MaxHR=108, ExerciseAngina='Y', Oldpeak=1.5, ST_Slope='Flat', HeartDisease=1),
 Row(Age=54, Gender='M', ChestPainType='NAP', RestingBP=150, Cholesterol=195, FastingBS=0, RestingECG='Normal', MaxHR=122, ExerciseAngina='N', Oldpeak=0.0, ST_Slope='Up', HeartDisease=0)]

In [16]:
df.groupBy('HeartDisease').count().show()

+------------+-----+
|HeartDisease|count|
+------------+-----+
|           1|  508|
|           0|  410|
+------------+-----+



In [17]:
df.groupBy('Age').count().show()

+---+-----+
|Age|count|
+---+-----+
| 31|    2|
| 65|   21|
| 53|   33|
| 34|    7|
| 28|    1|
| 76|    2|
| 44|   19|
| 47|   19|
| 52|   36|
| 40|   13|
| 57|   38|
| 54|   51|
| 48|   31|
| 64|   22|
| 41|   24|
| 43|   24|
| 37|   11|
| 61|   31|
| 72|    4|
| 35|   11|
+---+-----+
only showing top 20 rows



In [18]:
df.groupBy('Gender').count().show()

+------+-----+
|Gender|count|
+------+-----+
|     F|  193|
|     M|  725|
+------+-----+



In [19]:
df.groupBy('Age').mean().show()

+---+--------+------------------+------------------+-------------------+------------------+-------------------+-------------------+
|Age|avg(Age)|    avg(RestingBP)|  avg(Cholesterol)|     avg(FastingBS)|        avg(MaxHR)|       avg(Oldpeak)|  avg(HeartDisease)|
+---+--------+------------------+------------------+-------------------+------------------+-------------------+-------------------+
| 31|    31.0|             110.0|             244.5|                0.0|             151.5|               0.75|                0.5|
| 65|    65.0|141.76190476190476|207.57142857142858| 0.3333333333333333| 125.9047619047619| 1.1714285714285715| 0.7142857142857143|
| 53|    53.0| 131.0909090909091|181.03030303030303| 0.2727272727272727|131.57575757575756| 0.7181818181818181|0.45454545454545453|
| 34|    34.0|124.14285714285714|163.28571428571428|0.14285714285714285|172.57142857142858|0.12857142857142856| 0.2857142857142857|
| 28|    28.0|             130.0|             132.0|                0.0|    

In [20]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- ChestPainType: string (nullable = true)
 |-- RestingBP: integer (nullable = true)
 |-- Cholesterol: integer (nullable = true)
 |-- FastingBS: integer (nullable = true)
 |-- RestingECG: string (nullable = true)
 |-- MaxHR: integer (nullable = true)
 |-- ExerciseAngina: string (nullable = true)
 |-- Oldpeak: double (nullable = true)
 |-- ST_Slope: string (nullable = true)
 |-- HeartDisease: integer (nullable = true)



In [21]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [22]:
indexer=StringIndexer(inputCol= "Gender",outputCol='Gender_cat')
indexed=indexer.fit(df).transform(df)

In [23]:
indexer=StringIndexer(inputCol= "ChestPainType",outputCol='ChestPainType_cat')
indexed=indexer.fit(indexed).transform(indexed)

In [24]:
indexer=StringIndexer(inputCol= "RestingECG",outputCol='RestingECG_cat')
indexed=indexer.fit(indexed).transform(indexed)

In [25]:
indexer=StringIndexer(inputCol= "ExerciseAngina",outputCol='ExerciseAngina_cat')
indexed=indexer.fit(indexed).transform(indexed)

In [26]:
indexer=StringIndexer(inputCol= "ST_Slope",outputCol='ST_Slope_cat')
indexed=indexer.fit(indexed).transform(indexed)

In [27]:
indexed.show()

+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+----------+-----------------+--------------+------------------+------------+
|Age|Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|Gender_cat|ChestPainType_cat|RestingECG_cat|ExerciseAngina_cat|ST_Slope_cat|
+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+----------+-----------------+--------------+------------------+------------+
| 40|     M|          ATA|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|       0.0|              2.0|           0.0|               0.0|         1.0|
| 49|     F|          NAP|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|       1.0|              1.0|           0.0|               0.0|         0.0|
| 37|     M|   

In [28]:
indexed.columns

['Age',
 'Gender',
 'ChestPainType',
 'RestingBP',
 'Cholesterol',
 'FastingBS',
 'RestingECG',
 'MaxHR',
 'ExerciseAngina',
 'Oldpeak',
 'ST_Slope',
 'HeartDisease',
 'Gender_cat',
 'ChestPainType_cat',
 'RestingECG_cat',
 'ExerciseAngina_cat',
 'ST_Slope_cat']

In [37]:

assembler= VectorAssembler(inputCols=['Age',
 'RestingBP',
 'FastingBS',
 'MaxHR',
 'Oldpeak',
 'Gender_cat', 'ChestPainType_cat', 'RestingECG_cat', 'ST_Slope_cat'],outputCol='features')

In [38]:
assembler

VectorAssembler_bd8b30ff07d0

In [39]:
output= assembler.transform(indexed)

In [40]:
output

DataFrame[Age: int, Gender: string, ChestPainType: string, RestingBP: int, Cholesterol: int, FastingBS: int, RestingECG: string, MaxHR: int, ExerciseAngina: string, Oldpeak: double, ST_Slope: string, HeartDisease: int, Gender_cat: double, ChestPainType_cat: double, RestingECG_cat: double, ExerciseAngina_cat: double, ST_Slope_cat: double, features: vector]

In [41]:
output.show()

# As we can see, now we have one extra column named features, which is
#nothing but a combination of all the input features represented as a single dense
#vector.

+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+----------+-----------------+--------------+------------------+------------+--------------------+
|Age|Gender|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|Gender_cat|ChestPainType_cat|RestingECG_cat|ExerciseAngina_cat|ST_Slope_cat|            features|
+---+------+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+----------+-----------------+--------------+------------------+------------+--------------------+
| 40|     M|          ATA|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|       0.0|              2.0|           0.0|               0.0|         1.0|[40.0,140.0,0.0,1...|
| 49|     F|          NAP|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|       1

In [42]:
output.select('features','HeartDisease').show(5)

+--------------------+------------+
|            features|HeartDisease|
+--------------------+------------+
|[40.0,140.0,0.0,1...|           0|
|[49.0,160.0,0.0,1...|           1|
|[37.0,130.0,0.0,9...|           0|
|[48.0,138.0,0.0,1...|           1|
|[54.0,150.0,0.0,1...|           0|
+--------------------+------------+
only showing top 5 rows



In [43]:
indexed.columns

['Age',
 'Gender',
 'ChestPainType',
 'RestingBP',
 'Cholesterol',
 'FastingBS',
 'RestingECG',
 'MaxHR',
 'ExerciseAngina',
 'Oldpeak',
 'ST_Slope',
 'HeartDisease',
 'Gender_cat',
 'ChestPainType_cat',
 'RestingECG_cat',
 'ExerciseAngina_cat',
 'ST_Slope_cat']

In [44]:
model_df=output.select(['features','HeartDisease'])

In [45]:
training_df,test_df=model_df.randomSplit([0.75,0.25])

In [46]:
print(training_df.count())

694


In [47]:
print(test_df.count())

224


In [48]:
from pyspark.ml.classification import LogisticRegression

In [49]:
log_reg=LogisticRegression(labelCol='HeartDisease').fit(training_df)

23/04/30 15:30:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [50]:
# training results
train_results=log_reg.evaluate(training_df).predictions

In [51]:
train_results.show()

+--------------------+------------+--------------------+--------------------+----------+
|            features|HeartDisease|       rawPrediction|         probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
|(9,[0,1,2,3],[40....|           1|[-1.8729887098740...|[0.13319628249822...|       1.0|
|(9,[0,1,2,3],[48....|           1|[-2.8682488085099...|[0.05374564287109...|       1.0|
|(9,[0,1,2,3],[51....|           1|[-3.2061674734550...|[0.03893428837329...|       1.0|
|(9,[0,1,2,3],[51....|           1|[-3.0326790577465...|[0.04597118633683...|       1.0|
|(9,[0,1,2,3],[52....|           1|[-2.8243938655929...|[0.05602012395790...|       1.0|
|(9,[0,1,2,3],[53....|           1|[-2.8454523185199...|[0.05491686927709...|       1.0|
|(9,[0,1,2,3],[57....|           1|[-3.2415372788462...|[0.03763217683424...|       1.0|
|(9,[0,1,2,3],[59....|           1|[-3.3689177106564...|[0.03328111227036...|       1.0|
|(9,[0,1,3],[32.0,...

In [52]:
results=log_reg.evaluate(test_df).predictions

In [53]:
results.printSchema()

root
 |-- features: vector (nullable = true)
 |-- HeartDisease: integer (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [54]:
results.select(['HeartDisease','prediction']).show(10,False)

+------------+----------+
|HeartDisease|prediction|
+------------+----------+
|1           |1.0       |
|1           |1.0       |
|1           |1.0       |
|1           |1.0       |
|1           |1.0       |
|1           |1.0       |
|1           |1.0       |
|1           |1.0       |
|1           |1.0       |
|1           |1.0       |
+------------+----------+
only showing top 10 rows



In [55]:
# confusion matrix
tp = results[(results.HeartDisease == 1) & (results.prediction == 1)].count()

In [56]:
tp

108

In [57]:
tn = results[(results.HeartDisease == 0) & (results.prediction ==0)].count()

In [58]:
tn

81

In [59]:
fp = results[(results.HeartDisease == 0) & (results.prediction == 1)].count()

In [60]:
fp

22

In [61]:
fn = results[(results.HeartDisease == 1) & (results.prediction ==0)].count()

In [62]:
fn

13

In [63]:
# accuracy
accuracy=float((tp+tn)/(results.count()))

In [64]:
accuracy

0.84375